In [14]:
#import libraries
import fiona
import pandas as pd
import numpy as np

import os

#bokeh
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

#color
from bokeh.palettes import Reds6 as palette

from bokeh.layouts import layout, column, row, WidgetBox
from bokeh.models import Panel, Spacer, HoverTool, LogColorMapper, ColumnDataSource
from bokeh.models.widgets import Div, Tabs, Paragraph, Dropdown, Button, PreText, Toggle

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

from bokeh.resources import CDN

from bokeh.palettes import PuBu9

#mapping
from shapely.geometry import Polygon, Point, MultiPoint, MultiPolygon
from shapely.prepared import prep

from itertools import chain

import warnings
warnings.filterwarnings('ignore')

import math

from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import GraphRenderer, StaticLayoutProvider, Oval
from bokeh.palettes import Spectral8
from bokeh.tile_providers import STAMEN_TERRAIN_RETINA,CARTODBPOSITRON_RETINA

#import holoviews as hv
#hv.extension('bokeh')

from bokeh.models import Plot, ColumnDataSource, Range1d, from_networkx, Circle,MultiLine
from bokeh.io import show, output_file
from bokeh.palettes import Viridis

import geopandas as gpd

import networkx as nx

output_notebook()

ModuleNotFoundError: No module named 'networkx'

In [6]:
#data
cur = os.getcwd()

model = pd.read_csv(os.path.join(cur,'data','model_walktransit.csv'))
survey = pd.read_csv(os.path.join(cur,'data','TravelSurvey_weekday_rsg.csv'))

rings = os.path.join(cur,'data','shapefiles\ring_sector_prj3.shp')
rings_pts = pd.read_csv(os.path.join(cur,'data','ring_latlon3.csv'))
metra = os.path.join(cur,'data','shapefiles\metra_rail_Project.shp')
cta = os.path.join(cur,'data','shapefiles\cta_rail_Project.shp')

In [7]:
#transit flows
def crosstabs_pct(df, index, col, value):
    return (pd.crosstab(index=df[index],columns=df[col],
            values=df[value],aggfunc=sum,margins=True,
            margins_name='Total', normalize='all')*100)


def transit_flows(model,survey):

    def replace_label(df):
        replace = {17:"Illinois",19:"Wisconsin",18:"Indiana",14:"Kendall",12:"Kane",
           11:"DuPage",8:"Cook",7:"Suburban Cook",5:"Chicago",
           13:"Will",15:"McHenry",16:"Lake",9:"Suburban Cook",3:"Chicago",4:"Chicago",1:"Central Business District"}

        df.loc[:,'origAreaGroup'] = df['origAreaGroup'].replace(replace)
        df.loc[:,'destAreaGroup'] = df['destAreaGroup'].replace(replace)
        return df

    model_ct = crosstabs_pct(replace_label(model),'origAreaGroup','destAreaGroup','Model')
    survey_ct = crosstabs_pct(replace_label(survey),'origAreaGroup','destAreaGroup','Weight2')

    return model_ct.fillna(0).reset_index(), survey_ct.fillna(0).reset_index()

In [8]:
model_transit = model.loc[(model['transit_mode']!='Auto') & (~model['hhincome'].isin([0,'0']))]
survey_transit = survey.loc[(survey['transit_mode']!='Auto') & (~survey['hhincome'].isin([0,'0','na']))]

#transit flows
flows = transit_flows(model_transit,survey_transit)
model_flows = flows[0]
survey_Flows = flows[1]

In [9]:
def make_map_src(shapefile,label,):
    shp = fiona.open(shapefile)
    # Extract features from shapefile
    district_name = [ feat["properties"][label] for feat in shp]
    district_area = [ feat["properties"]["Shape_Area"] for feat in shp]
    district_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp]
    district_y = [ [y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp]
    district_xy = [ [ xy for xy in feat["geometry"]["coordinates"][0]] for feat in shp] 
    district_poly = [ Polygon(xy) for xy in district_xy] # coords to Polygon

    source = ColumnDataSource(data=dict(
        x=district_x, y=district_y,
        name=district_name
    ))

    return source

def make_map(src,points):
        
        TOOLS = "pan,wheel_zoom,reset,hover,save"
        
        p = figure(tools=TOOLS, width=800,height=800,
            x_axis_location=None, y_axis_location=None
        )
        p.grid.grid_line_color = None
        
        p.patches('x', 'y', source=src,
                  fill_alpha=None, line_color='Black', line_width=0.3)
        
        hover = p.select_one(HoverTool)
        hover.point_policy = "follow_mouse"
        hover.tooltips = [("zones", "@name")]
        
        p.circle(x='lon',y='lat', size=5, fill_color="blue", fill_alpha=0.8, source=points)
    

        return p.patches('x', 'y', source=src,
                  fill_alpha=None, line_color='Black', line_width=0.3)
    
def make_map_1(src,title,v):

    custom_colors = ['#f2f2f2', '#fee5d9', '#fcbba1', '#fc9272', '#fb6a4a', '#de2d26']
    color_mapper = LogColorMapper(palette=custom_colors)   

    TOOLS = "pan,wheel_zoom,reset,hover,save"

    p = figure(
        title=title, tools=TOOLS, width=800,height=800,
        x_axis_location=None, y_axis_location=None,x_range=v.x_range, y_range=v.y_range
    )
    p.grid.grid_line_color = None
    p.patches('x', 'y', source=src,
              fill_color={'field': 'rate', 'transform': color_mapper},
              fill_alpha=0.8, line_color=None, line_width=0.3)

    hover = p.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [("zones", "@name"),("trips", "@rate")]

    p = style(p)

    return p

In [13]:
node_initial_pos = {}
for i, row in rings_pts.iterrows():
    node_initial_pos[rings_pts.loc[i,'rs2']] = (rings_pts.loc[i,'POINT_X'],rings_pts.loc[i,'POINT_Y'])
    
xmin = np.min(rings_pts['POINT_X'].values.tolist())
xmax = np.max(rings_pts['POINT_X'].values.tolist())
ymin = np.min(rings_pts['POINT_Y'].values.tolist())
ymax = np.max(rings_pts['POINT_Y'].values.tolist())

#define graph
od = model.groupby(['origin_geo','destination_geo']).agg({'Model':sum}).reset_index()
od['color'] = pd.cut(od['Model'],9,labels =PuBu9 )
od.columns = ['source','target','weight','color']
G=nx.from_pandas_edgelist(od,edge_attr =True) # function signature changes
G.nodes

NameError: name 'nx' is not defined